# Curriculum Learning Training for Novel Writer
Train Llama-3 using curriculum learning - starting with easy examples and progressively increasing difficulty.

In [ ]:
!pip install -q unsloth peft transformers accelerate bitsandbytes datasets

In [ ]:
import json
from pathlib import Path
from datasets import Dataset
from transformers import TrainingArguments
from unsloth import FastLanguageModel
from trl import SFTTrainer

## Load Curriculum-Sorted Dataset

In [ ]:
# Load the curriculum-sorted dataset (easy to hard)
DATA_PATH = "data/processed/train_curriculum.jsonl"

entries = []
with open(DATA_PATH, "r") as f:
    for line in f:
        data = json.loads(line.strip())
        entries.append(data)

print(f"Loaded {len(entries)} entries")
print(f"Difficulty range: {entries[0].get('_difficulty', 'N/A')} to {entries[-1].get('_difficulty', 'N/A')}")

## Split into Curriculum Stages

In [ ]:
# Split into 3 curriculum stages
n = len(entries)
stage1 = entries[:n//3]          # Easy
stage2 = entries[n//3:2*n//3]    # Medium
stage3 = entries[2*n//3:]        # Hard

stages = [
    ("Stage 1: Easy", stage1),
    ("Stage 2: Medium", stage2),
    ("Stage 3: Hard", stage3),
]

for name, data in stages:
    print(f"{name}: {len(data)} examples")

## Load Model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=8192,
    dtype=None,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0,
    bias="none",
)

## Curriculum Training Loop

In [ ]:
def format_entry(entry):
    instruction = entry.get("instruction", "Continue the story:")
    output = entry.get("output", entry.get("text", ""))
    return f"### Instruction:\n{instruction}\n\n### Response:\n{output}"

for stage_name, stage_data in stages:
    print(f"\n{'='*50}")
    print(f"Training {stage_name} ({len(stage_data)} examples)")
    print(f"{'='*50}\n")
    
    # Format dataset
    formatted = [{"text": format_entry(e)} for e in stage_data]
    dataset = Dataset.from_list(formatted)
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        args=TrainingArguments(
            output_dir=f"curriculum_output/{stage_name.lower().replace(' ', '_').replace(':', '')}",
            num_train_epochs=1,
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            learning_rate=2e-4,
            logging_steps=10,
            save_steps=100,
            bf16=True,
        ),
    )
    
    trainer.train()
    print(f"Completed {stage_name}")

## Save Final Model

In [ ]:
model.save_pretrained("curriculum_lora_model")
tokenizer.save_pretrained("curriculum_lora_model")
print("Curriculum-trained model saved to curriculum_lora_model/")